In [ ]:
#%matplotlib inline
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
np.random.seed(0)
import pickle
#plt.style.use("ggplot")
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


##Data preprocessing 

In [ ]:
data = pd.read_csv('ROM_withiut_Tag.csv')
#data = data.fillna(method = 'ffill')
data.head()

,Sentence #,Word,POS,Agila_DB_tag
0,1,Read,VERB,NaN
1,1,-,PUNCT,NaN
2,1,only,ADV,NaN
3,1,memory,NOUN,NaN
4,1,(,PUNCT,NaN


In [ ]:
data['Sentence #'].value_counts()

42     95
37     75
28     75
88     69
75     68
       ..
33     10
82     10
131    10
110     8
132     5
Name: Sentence #, Length: 135, dtype: int64

In [ ]:
print('unique words in corpus:', data['Word'].nunique())

unique words in corpus: 1155


In [ ]:
class SentenceGetter(object):
    """Retrieves sentences and their corresponding tags."""
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        aggregate_function = lambda s: [(w, p,t) for w, p,t in zip(s['Word'].values.tolist(),
                                                                    s['POS'].values.tolist(),
                                                                    s['Agila_DB_tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(aggregate_function)
        self.sentences = [s for s in self.grouped]

In [ ]:
getter = SentenceGetter(data)

In [ ]:
sentences = getter.sentences

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")

### Loading our tag to index from save file for our new dataset

In [ ]:
with open('/content/drive/My Drive/Thesis/Pickle file/word_to_indexCombinedDataForTest.pkl', 'rb') as f:
    word_to_index = pickle.load(f)

In [ ]:
with open('/content/drive/My Drive/Thesis/Pickle file/tag_to_indexCombinedDataForTest.pkl', 'rb') as f:
    tag_to_index = pickle.load(f)

##Getting the new words and tags for new dataset

In [ ]:
words = [k for k in word_to_index.keys()]
num_words = len(words)

In [ ]:
tags = [k for k in tag_to_index.keys()]
num_tags = len(tags)

In [ ]:
print(word_to_index)

{'consuming': 1, 'reconstruct': 2, 'compresses': 3, 'leading': 4, 'numeric': 5, 'losses': 6, 'price': 7, 'Gigabit': 8, 'contamination': 9, 'Hardware': 10, '–': 11, 'partition': 12, 'variously': 13, 'strong': 14, 'measuring': 15, 'car': 16, 'MT': 17, 'things': 18, 'floppy': 19, 'Hyperthreading': 20, 'subject': 21, 'Temperature': 22, 'Super': 23, 'gas': 24, 'machines': 25, 'trivial': 26, 'disc': 27, 'Use': 28, 'pack': 29, 'sharper': 30, 'uninterruptible': 31, 'includes': 32, 'controlled': 33, 'ripples': 34, 'poorly': 35, 'treats': 36, 'Full': 37, 'Arithmetic': 38, '15': 39, 'CORDIC': 40, 'bit)Many': 41, 'consume': 42, 'Most': 43, 'add': 44, 'type': 45, 'temporary': 46, 'forward': 47, 'transferring': 48, 'M8C': 49, '1953': 50, 'session': 51, 'document': 52, 'anti': 53, '3.3': 54, 'interface': 55, 'situations': 56, 'complied': 57, 'performs': 58, 'effectively': 59, 'unwanted': 60, 'appropriate': 61, 'ball': 62, 'Motor': 63, '620': 64, 'batteries': 65, '550': 66, 'film': 67, 'transactional'

In [ ]:
print(tag_to_index)

{'B-func': 0, 'I-sys': 1, 'I-hwc': 2, 'B-hwc': 3, 'I-sw': 4, 'I-hwp': 5, 'B-sw': 6, 'I-mea': 7, 'B-qt': 8, 'B-hwp': 9, 'B-unit': 10, 'I-hwsp': 11, 'I-func': 12, 'B-hwsp': 13, 'I-comp': 14, 'O': 15, 'I-qt': 16, 'B-comp': 17, 'B-mea': 18, 'I-unit': 19, 'B-sys': 20}


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

max_len = 50

* Taking sentences from our new dataset and convert each of the words of the sentences to our trained index. as we trained our model with index number we have to give index number again when predictiong new data.

#preparing our test dataset X

In [ ]:
X = [[word_to_index[w[0]] for w in s if w[0] in word_to_index.keys()] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding='post', value=num_words-1)
#we are not using any tag here
# y = [[tag2index[w[2]] for w in s] for s in sentences]
# y = pad_sequences(maxlen=max_len, sequences=y, padding='post', value=tag2index["O"])
# y = [to_categorical(i, num_classes=num_tags) for i in y]

> same shape as x_train of our model

In [ ]:
X.shape

(375, 50)

In [ ]:
from tensorflow.keras.models import load_model
new_model = load_model('/content/drive/My Drive/Thesis/model save/KB_LSTM_model_C8_50.h5')

In [ ]:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding (Embedding)       (None, 50, 50)            327100    
                                                                 
 spatial_dropout1d (SpatialD  (None, 50, 50)           0         
 ropout1D)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 50, 512)          628736    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 50, 21)           10773     
 ibuted)                                                         
                                                             

## Predicting new data with our trained model. will pass X[i] as our "data to be predicted" which is basically index number. then convert it to word using for loop.

In [ ]:
list_words = []
list_tags_true =[]
list_tags_pred =[]

for i in range(X.shape[0]):
  preds = new_model.predict(np.array([X[i]])) ##passing indexes of our new data
  pred = np.argmax(preds, axis=-1)
  for w,p in zip(X[i],pred[0]):
    list_words.append(words[w-1])
    list_tags_pred.append(tags[p])
table = pd.DataFrame({'Word':list_words,'Pred':list_tags_pred})
#print(table.to_string())
table.to_csv('/content/drive/My Drive/Thesis/Training_result/ROM_prediction.csv')